In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import joblib
import os

def QNNs_gen(data, target_column, feature_column, save_path, number_of_quantile_one_end=4):
    def ensure_directory_exists(path):
        if not os.path.exists(path):
            os.makedirs(path)
            print(f"Created directory: {path}")
        else:
            print(f"Directory already exists: {path}")
    ensure_directory_exists(save_path)

    # Define grid search parameters
    layer_configs = [[128, 64, 32], [256, 128, 64, 32]]
    learning_rates = [0.01, 0.005, 0.001]
    optimizers = [optim.Adam, optim.SGD]

    class NeuralNet(nn.Module):
        def __init__(self, input_size, output_size, layers):
            super(NeuralNet, self).__init__()
            self.layers = nn.ModuleList()
            self.activations = nn.ModuleDict()
            last_size = input_size
            for i, size in enumerate(layers):
                self.layers.append(nn.Linear(last_size, size))
                self.activations[f'layer{i+1}'] = nn.ReLU()
                last_size = size
            self.output_layer = nn.Linear(last_size, output_size)

        def forward(self, x):
            for i, linear in enumerate(self.layers):
                x = linear(x)
                x = self.activations[f'layer{i+1}'](x)
            return self.output_layer(x)

    # Prepare data
    features = data[feature_column]
    target = data[target_column]
    X = torch.tensor(features.values, dtype=torch.float32)
    y = torch.tensor(target.values, dtype=torch.float32).unsqueeze(1)
    scaler = StandardScaler()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    X_train = torch.tensor(scaler.fit_transform(X_train), dtype=torch.float32)
    X_test = torch.tensor(scaler.transform(X_test), dtype=torch.float32)
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

    best_avg_loss = float('inf')
    best_model = None

    # Grid search loop
    for layers in layer_configs:
        for lr in learning_rates:
            for Optimizer in optimizers:
                quantiles = [0.5 + (i / (number_of_quantile_one_end * 2 + 2)) for i in range(-number_of_quantile_one_end, number_of_quantile_one_end + 1)]
                models = [NeuralNet(X_train.shape[1], 1, layers) for _ in quantiles]
                optimizers = [Optimizer(model.parameters(), lr=lr) for model in models]

                for epoch in range(200):
                    epoch_losses = []
                    for inputs, labels in train_loader:
                        current_outputs = {}
                        for model, optimizer, quantile in zip(models, optimizers, quantiles):
                            optimizer.zero_grad()
                            outputs = model(inputs)
                            current_outputs[quantile] = outputs.detach()

                            # Compute bounds and quantile loss
                            lower_bound, upper_bound = set_bounds(quantiles, current_outputs, quantile)
                            loss = adjusted_quantile_loss(outputs, labels, quantile, lower_bound, upper_bound)
                            loss.backward()
                            optimizer.step()
                            epoch_losses.append(loss.item())

                        current_outputs.clear()

                    avg_loss = np.mean(epoch_losses)
                    if avg_loss < best_avg_loss:
                        best_avg_loss = avg_loss
                        best_model = {'model': models, 'layers': layers, 'lr': lr, 'optimizer': Optimizer.__name__}

                # Evaluate and print R-squared after training each configuration
                r2_scores = [r2_score(y_test.numpy(), model(X_test).detach().numpy()) for model in models]
                print(f'Configuration: Layers: {layers}, LR: {lr}, Optimizer: {Optimizer.__name__}, R2 Scores: {r2_scores}')

    # Save the best model
    model_save_path = os.path.join(save_path, 'best_model.pth')
    torch.save({'state_dict': best_model['model'][0].state_dict(), 'config': best_model}, model_save_path)

    return best_model['model'][0]

# Example usage remains as previously described.

def adjusted_quantile_loss(outputs, targets, quantile, lower_bound=None, upper_bound=None):
    errors = targets - outputs
    basic_loss = torch.max((quantile - 1) * errors, quantile * errors)
    loss = torch.mean(basic_loss)

    if lower_bound is not None:
        crossing_penalty = torch.mean(torch.relu(lower_bound - outputs))
        loss += crossing_penalty
    if upper_bound is not None:
        crossing_penalty = torch.mean(torch.relu(outputs - upper_bound))
        loss += crossing_penalty

    return loss

def set_bounds(quantiles, current_outputs, current_quantile):
    lower_bound = None
    upper_bound = None
    idx = quantiles.index(current_quantile)
    if idx > 0:
        lower_bound = current_outputs.get(quantiles[idx - 1])
    if idx < len(quantiles) - 1:
        upper_bound = current_outputs.get(quantiles[idx + 1])
    return lower_bound, upper_bound

In [3]:
# Example usage with your data loading
data = pd.read_csv('/home/yui/Downloads/read_and_play/p122_synthetic_1_1.1_1.2.csv')  # Load your data
feature_column = ['battery_2#p122', 'gas-cc#p122', 'upv#p122', 'wind-ons#p122']
target_column = 'value'
save_path = "model_folder"
best_model = QNNs_gen(data, target_column, feature_column, save_path, number_of_quantile_one_end=4)

Directory already exists: model_folder
Epoch 0, Quantile 0.09999999999999998, Train R2: -0.4124, Test R2: -1.0542
Epoch 0, Quantile 0.2, Train R2: -0.4124, Test R2: -1.0542
Epoch 0, Quantile 0.3, Train R2: -0.4124, Test R2: -1.0542
Epoch 0, Quantile 0.4, Train R2: -0.4124, Test R2: -1.0542
Epoch 0, Quantile 0.5, Train R2: -0.4124, Test R2: -1.0542
Epoch 0, Quantile 0.6, Train R2: -0.4124, Test R2: -1.0542
Epoch 0, Quantile 0.7, Train R2: -0.4124, Test R2: -1.0542
Epoch 0, Quantile 0.8, Train R2: -0.4124, Test R2: -1.0542
Epoch 0, Quantile 0.9, Train R2: -0.4124, Test R2: -1.0542
Epoch 0, Quantile 0.09999999999999998, Train R2: -1.2540, Test R2: -1.0542
Epoch 0, Quantile 0.2, Train R2: -1.2540, Test R2: -1.0542
Epoch 0, Quantile 0.3, Train R2: -1.2540, Test R2: -1.0542
Epoch 0, Quantile 0.4, Train R2: -1.2540, Test R2: -1.0542
Epoch 0, Quantile 0.5, Train R2: -1.2540, Test R2: -1.0542
Epoch 0, Quantile 0.6, Train R2: -1.2540, Test R2: -1.0542
Epoch 0, Quantile 0.7, Train R2: -1.2540, Te

KeyboardInterrupt: 